In [1]:
!pip install -U datasets==2.17.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 38.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.0
    Uninstalling fsspec-2024.3.0:
      Successfully uninstalled fsspec-2024.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-

In [2]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1', download_mode="force_redownload")

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
datasets["train"][10]

In [3]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [4]:
show_random_elements(datasets["train"])

,text
0,"The album was recorded live at the Holland Festival on June 18 , 2010 , in the Muziekgebouw aan ' t IJ . The work was performed by Mike Patton and the Belgian Ictus Ensemble conducted by Georges @-@ Elie Octors . Solos were performed by Ictus Ensemble clarinetist Dirk Descheemaeker , trumpeter Loïc Dumoulin , trombonist Michel Massot , double bass player Géry Cambier , and percussionists Michael Weilacher and Gerrit Nulens . Nederlands Kamerkoor provided the choral accompaniments . The album marks only the third recording of the composition to have been released since it was first broadcast on French radio by Office de Radiodiffusion Télévision Française . Patton has said of the work , "" I can listen to Berio and Nono as easily as I can to Morricone but like all modern music of Italy , it is unfortunately marginalized ... Maybe because of the language barrier , maybe because it ’ s not easily understood . Berio , who was teaching in California when he wrote this piece , was listening to jazz , pop and folk music and incorporated all of it in his works without prejudice . "" \n"
1,
2,"In 1993 , Shimomura left Capcom to join another game company , Square ( now Square Enix ) . She stated that the move was done because she was interested in writing "" classical @-@ style "" music for fantasy role @-@ playing games . While working for Capcom , she was in the arcade department and was unable to transfer to the console department to work on their role @-@ playing video game series Breath of Fire , although she did contribute one track to the first game in the series . Her first project at the company was the score for the role @-@ playing video game Live A Live in 1994 . While she was working on the score to Super Mario RPG the following year , she was asked to join Noriko Matsueda on the music to the futuristic strategy RPG Front Mission . Although she was overworked doing both scores and it was not the genre that she was interested in , she found herself unable to refuse after her first attempt to do so unexpectedly happened in the presence of the president of Square , Tetsuo Mizuno . These games were followed by Tobal No. 1 , the last score she worked on with another composer for a decade . \n"
3,"Palace Software , the developer of the two Barbarian games , marketed the sequel with the same strategy they used for the first game . They hired Maria Whittaker , a model known for her topless work , to pose on the cover and posters as the princess in the game , attempting to recapture the controversy that had boosted sales . Barbarian II received a mixed critical reception . Reviewers were split in their opinions over whether the game was a refreshing and gory adventure , or a boring and lonely sojourn through a confusing digital world . \n"
4,"Coldrum Long Barrow is comparatively isolated from the other Medway Megaliths ; in this it is unique , given that the other surviving examples are clustered into two groups . However , it is possible that another chambered tomb was located nearby ; a razed , elongated earthen mound with an east @-@ west orientation is located in a hollow at the foot of the downs just under a quarter of a mile to the north of the Coldrum Stones . It may be that this represents the remnants of another such monument which has had its stones removed or buried . Several large sarsens to the south of the Coldrums might represent the remnants of a further such tomb , since destroyed . \n"
5,= = = Other championships = = = \n
6,
7,"The noisy miner has some of the largest group sizes of any communally breeding bird , with up to twenty males and one female attending a single brood . Only males help with a nest , and while many birds may be associated with a particular brood , some males devote all their time to a single nest , while others spread their helping efforts across five or six nests . Behavioural evidence and genetic testing indicate that helpers are male offspring of the breeding pair , or full siblin

## Causal Language modeling
For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:

part of text 1
or

end of text 1 [BOS_TOKEN] beginning of text 2
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

In [5]:
from transformers import AutoTokenizer
model_checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [7]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3760
    })
})

In [9]:
block_size = 128

In [10]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [11]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [12]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

2024-04-11 15:53:26.164867: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 15:53:26.165000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 15:53:26.274753: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


' game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Oz'

In [13]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, device_map="auto")

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
from transformers import Trainer, TrainingArguments

In [15]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
#     push_to_hub=True,
)

In [16]:
lm_datasets["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 18666
})

In [17]:
lm_datasets["train"]['input_ids'][0]

[796,
 569,
 18354,
 7496,
 17740,
 6711,
 796,
 220,
 198,
 2311,
 73,
 13090,
 645,
 569,
 18354,
 7496,
 513,
 1058,
 791,
 47398,
 17740,
 357,
 4960,
 1058,
 10545,
 230,
 99,
 161,
 254,
 112,
 5641,
 44444,
 9202,
 25084,
 24440,
 12675,
 11839,
 18,
 837,
 6578,
 764,
 569,
 18354,
 7496,
 286,
 262,
 30193,
 513,
 1267,
 837,
 8811,
 6412,
 284,
 355,
 569,
 18354,
 7496,
 17740,
 6711,
 2354,
 2869,
 837,
 318,
 257,
 16106,
 2597,
 2488,
 12,
 31,
 2712,
 2008,
 983,
 4166,
 416,
 29490,
 290,
 6343,
 13,
 44206,
 329,
 262,
 14047,
 44685,
 764,
 28728,
 287,
 3269,
 2813,
 287,
 2869,
 837,
 340,
 318,
 262,
 2368,
 983,
 287,
 262,
 569,
 18354,
 7496,
 2168,
 764,
 12645,
 278,
 262,
 976,
 21748,
 286,
 16106,
 290,
 1103,
 2488,
 12,
 31,
 640,
 11327,
 355,
 663,
 27677,
 837,
 262,
 1621,
 4539,
 10730,
 284,
 262,
 717]

In [18]:
lm_datasets["train"]['labels'][0]

[796,
 569,
 18354,
 7496,
 17740,
 6711,
 796,
 220,
 198,
 2311,
 73,
 13090,
 645,
 569,
 18354,
 7496,
 513,
 1058,
 791,
 47398,
 17740,
 357,
 4960,
 1058,
 10545,
 230,
 99,
 161,
 254,
 112,
 5641,
 44444,
 9202,
 25084,
 24440,
 12675,
 11839,
 18,
 837,
 6578,
 764,
 569,
 18354,
 7496,
 286,
 262,
 30193,
 513,
 1267,
 837,
 8811,
 6412,
 284,
 355,
 569,
 18354,
 7496,
 17740,
 6711,
 2354,
 2869,
 837,
 318,
 257,
 16106,
 2597,
 2488,
 12,
 31,
 2712,
 2008,
 983,
 4166,
 416,
 29490,
 290,
 6343,
 13,
 44206,
 329,
 262,
 14047,
 44685,
 764,
 28728,
 287,
 3269,
 2813,
 287,
 2869,
 837,
 340,
 318,
 262,
 2368,
 983,
 287,
 262,
 569,
 18354,
 7496,
 2168,
 764,
 12645,
 278,
 262,
 976,
 21748,
 286,
 16106,
 290,
 1103,
 2488,
 12,
 31,
 640,
 11327,
 355,
 663,
 27677,
 837,
 262,
 1621,
 4539,
 10730,
 284,
 262,
 717]

In [19]:
lm_datasets["validation"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1931
})

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [21]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,3.751200,3.665518
2,3.648300,3.646266


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
test_input[:7]

[983, 290, 5679, 262, 366, 17871, 5321]

In [33]:
input_text = "game and follows the Nameless, a penal military unit"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

In [36]:
outputs = model.generate(**input_ids, max_new_tokens=60)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [37]:
tokenizer.decode(outputs[0])

'game and follows the Nameless, a penal military unit that is tasked with defending the city. The unit is tasked with defending the city, and is tasked with defending the city. The unit is tasked with defending the city, and is tasked with defending the city. The unit is tasked with defending the city, and is tasked with defending the city. The'